## ICU Labevents

The code below keeps only the labevents rows from patients with disease we have declare to the csv file

Export file -> CSV\Exports\o04_icu_chartevent.csv

------------------

I apply the chunk command because the file contains 59,327,830 lines which causes the computer to run out of memory.

I have set the size of the chunksize variable it will read to 100,000 rows. I set it according to the available computer memory.

In [ ]:
import gc
import gzip
import pandas as pd
import numpy as np

#Read the first_rows_df CSV file
first_stay_df = pd.read_csv('CSV\Exports\o03_icu_first_stay.csv')

# Read the chartevents CSV file in chunks
chunksize = 100000  # Number of rows to read in each chunk
labevents_df = pd.read_csv(r"..\Datasets\mimic-iv-2_2\hosp\labevents.csv.gz", low_memory=False, chunksize=chunksize)

# Create a list to store the processed chunks
processed_chunks = []

# Iterate over the chunks and process them
for chunk in labevents_df:
    # Merge the first_stay_df and chartevents_df DataFrames on the stay_id column
    merged_df = pd.merge(first_stay_df, chunk, on='hadm_id')

    # Add the processed chunk to the list
    processed_chunks.append(merged_df)

# Combine the processed chunks into a single DataFrame
labevents_df = pd.concat(processed_chunks)

# Drop the columns subject_id_y, hadm_id_y
labevents_df = labevents_df.drop(columns=['subject_id_y'])

# Rename the columns subject_id_x to subject_id and hamd_id_x to hadm_id
labevents_df = labevents_df.rename(columns={'subject_id_x': 'subject_id'})

# Keeping the rows that contain values in the valuenum
labevents_df = labevents_df[labevents_df['valuenum'].notna()]

In [ ]:
"""
Keeping the lab measurements
that have been requested during
the period stay in the ICU.
"""
# Convert 'charttime', 'intime', and 'outtime' columns to datetime objects
labevents_df['charttime'] = pd.to_datetime(labevents_df['charttime'])
labevents_df['intime'] = pd.to_datetime(labevents_df['intime'])
labevents_df['outtime'] = pd.to_datetime(labevents_df['outtime'])

# Now 'filtered_df' contains only the rows where 'charttime' is between 'intime' and 'outtime'
filtered_df = labevents_df[(labevents_df['charttime'] >= labevents_df['intime']) & (labevents_df['charttime'] <= labevents_df['outtime'])]

# Save chartevent dataset to a CSV file
filtered_df.to_csv('CSV\Exports\o04_icu_labevents.csv', index=False)

# Free RAM
gc.collect()